#Produkcja okien
Pewna firma produkuje dwa rodaje okien: aluminiowe i drewniane. Dysponuje trzema fabrykami, dwie z nich produkują odpowiednio ramy aluminiowe i drewniane. Trzecia dostarcza szyby - element wspólny dla okien. Jak należałoby zaplanować produkcję, aby zmaksymalizować zysk?

| Kategoria    | Okna aluminiowe | Okna drewniane | Dostępność |
|--------------|-----------------|----------------|------------|
| Praca fab. 1 | <center>1h | <center>0h | <center>4h/tydz.    |
| Praca fab. 2 | <center>0h | <center>2h | <center>12h/tydz.   |
| Praca fab. 3 | <center>3h | <center>2h | <center>18h/tydz.   |
| Zysk/1szt.   | <center>3000 | <center>5000 | <center>Max???     |



In [ ]:
!pip install pulp
!sudo apt-get install coinor-cbc glpk-utils coinor-clp

##Podejście pojedynczych zmiennych

In [2]:
from pulp import *

In [3]:
# Deklaracja problemu
ProdOkien = LpProblem("Maksymalizacja_zyskow_okien", LpMaximize)
# Deklaracja zmiennych decyzyjnych
IlOkienAlu = LpVariable("LiczbaPartii_Okien_Aluminiowych", 0, None, LpInteger)
IlOkienDrew = LpVariable("LiczbaPartii_Okien_Drewnianych", 0, None, LpInteger)
# Określenie funkcji optymalizowanej (zysku)
ProdOkien += IlOkienAlu * 3000 + IlOkienDrew * 5000, "Liczb_partii_wszystkich_okien"
# Określenie ograniczeń
ProdOkien += IlOkienAlu * 1 <= 4, "Limit_fabryki 1"
ProdOkien += IlOkienDrew * 2 <= 12, "Limit_fabryki_2"
ProdOkien += IlOkienAlu * 3 + IlOkienDrew * 2 <= 18, "Limit_fabryki_3"
# Roziwązanie problemu
ProdOkien.solve()
# Pozkazywanie wyniku rozwiązania
print("Status rozwiazania: ", LpStatus[ProdOkien.status])
print("Nalezy wyprodukowac: ")
for zm in ProdOkien.variables():
    print(zm.varValue, zm.name[zm.name.index("_") + 1:])


Status rozwiazania:  Optimal
Nalezy wyprodukowac: 
2.0 Okien_Aluminiowych
6.0 Okien_Drewnianych


## Podejście zmiennych grupowych

In [4]:
from pulp import *

In [5]:
# Deklaracja problemu
ProdOkien = LpProblem("Maksymalizacja_zyskow_okien", LpMaximize)

MaterialyOkien = ['DREWNO', 'ALUMINIUM']
Zysk = {
    'DREWNO':    5,
    'ALUMINIUM': 3
}
Fabr1 = {
    'DREWNO':    0,
    'ALUMINIUM': 1
}
Fabr2 = {
    'DREWNO':    2,
    'ALUMINIUM': 0
}
Fabr3 = {
    'DREWNO':    2,
    'ALUMINIUM': 3
}

WielkoscProd = LpVariable.dicts("IlPartiiMaterialu", MaterialyOkien, 0, None, LpInteger)
# Określenie funkcji optymalizowanej (zysku)
ProdOkien += lpSum([WielkoscProd[mat] * Zysk[mat] for mat in MaterialyOkien]), "Zysk"
# Określenie ograniczeń
ProdOkien += lpSum([Fabr1[mat] * WielkoscProd[mat] for mat in MaterialyOkien]) <= 4
ProdOkien += lpSum([Fabr2[mat] * WielkoscProd[mat] for mat in MaterialyOkien]) <= 12
ProdOkien += lpSum([Fabr3[mat] * WielkoscProd[mat] for mat in MaterialyOkien]) <= 18
# Roziwązanie problemu
ProdOkien.solve()
# Pozkazywanie wyniku rozwiązania
print("Status rozwiazania: ", LpStatus[ProdOkien.status])
print("Nalezy wyprodukowac: ")
for zm in ProdOkien.variables():
    print(zm.varValue, zm.name[zm.name.index("_") + 1:])

Status rozwiazania:  Optimal
Nalezy wyprodukowac: 
2.0 ALUMINIUM
6.0 DREWNO


#Wersja z edytowalnymi parametrami

In [6]:
# Import potrzebnych funkcji
from pulp import *
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual, Layout, FloatSlider, IntSlider
import ipywidgets as widgets
import pandas as pd

In [7]:
styl = {'description_width': 'initial'}

#suw_zysk_drew = FloatSlider(min = 0, max = 10000, step = 50, value = 5000, description =  "Zysk (drew.) [zł]", readout_format = '.0f', style = styl)
#suw_zysk_alu = FloatSlider(min = 0, max = 10000,  step = 50, value = 3000,  description = "Zysk (alum.) [zł]", readout_format = '.0f', style = styl)
suw_zysk_drew = IntSlider(min = 0, max = 10000, step = 50, value = 5000, description =  "Zysk (drew.) [zł]", readout_format = '.0f', style = styl)
suw_zysk_alu = IntSlider(min = 0, max = 10000,  step = 50, value = 3000,  description = "Zysk (alum.) [zł]", readout_format = '.0f', style = styl)
suw_fabr_1_drew = IntSlider(min = 0, max = 24, value = 0, description = "Czas prod. fab. 1 (drew.) [h]", style = styl)
suw_fabr_1_alu = IntSlider(min = 0, max = 24, value = 1,  description = "Czas prod. fab. 1 (alum.) [h]", style = styl)
suw_fabr_2_drew = IntSlider(min = 0, max = 24, value = 2, description = "Czas prod. fab. 2 (drew.) [h]", style = styl)
suw_fabr_2_alu = IntSlider(min = 0, max = 24, value = 0,  description = "Czas prod. fab. 2 (alum.) [h]", style = styl)
suw_fabr_3_drew = IntSlider(min = 0, max = 24, value = 2, description = "Czas prod. fab. 3 (drew.) [h]", style = styl)
suw_fabr_3_alu = IntSlider(min = 0, max = 24, value = 3,  description = "Czas prod. fab. 3 (alum.) [h]", style = styl)
suw_dostep_fabr_1 = IntSlider(min = 0, max = 24 * 7, value = 4,  description = "Dostęp. fab. 1 [h]", style = styl)
suw_dostep_fabr_2 = IntSlider(min = 0, max = 24 * 7, value = 12, description = "Dostęp. fab. 2 [h]", style = styl)
suw_dostep_fabr_3 = IntSlider(min = 0, max = 24 * 7, value = 18, description = "Dostęp. fab. 3 [h]", style = styl)

def wyznOptymPraceFabryk(
    ZyskOkDrew = 5000, ZyskOkAlu = 3000,
    PracaF1Drew = 0, PracaF1Alu = 1,
    PracaF2Drew = 2, PracaF2Alu = 0,
    PracaF3Drew = 2, PracaF3Alu = 3,
    DostepF1 = 4, DostepF2 = 12, DostepF3 = 18
):
    
    # Deklaracja problemu
    ProdOkien = LpProblem("Maksymalizacja_zyskow_okien", LpMaximize)
    # Deklaracja zmiennych decyzyjnych
    IlOkienAlu = LpVariable("LiczbaPartii_Okien_Aluminiowych", 0, None, LpContinuous)
    IlOkienDrew = LpVariable("LiczbaPartii_Okien_Drewnianych", 0, None, LpContinuous)
    # Określenie funkcji optymalizowanej (zysku)
    ProdOkien += IlOkienAlu * ZyskOkAlu + IlOkienDrew * ZyskOkDrew, "Liczb_partii_wszystkich_okien"
    # Określenie ograniczeń
    ProdOkien += IlOkienAlu * PracaF1Alu + IlOkienDrew * PracaF1Drew <= DostepF1, "Limit_fabryki 1"
    ProdOkien += IlOkienAlu * PracaF2Alu + IlOkienDrew * PracaF2Drew <= DostepF2, "Limit_fabryki_2"
    ProdOkien += IlOkienAlu * PracaF3Alu + IlOkienDrew * PracaF3Drew <= DostepF3, "Limit_fabryki_3"
    # Roziwązanie problemu
    ProdOkien.solve()
    # Pozkazywanie wyniku rozwiązania
    print("Status rozwiązania: ", LpStatus[ProdOkien.status])
    print("Zysk wyniesie: {:.2f} zł".format(ProdOkien.objective.value()) )
    print("Należy wyprodukować: ")
    for zm in ProdOkien.variables():
        print("{:.0f} {}".format(zm.varValue, zm.name[zm.name.index("_") + 1:]))
    tabWrazliwosci = [{"Nazwa":nazwa, "Wpływ": zm.pi, "Zapas": zm.slack} for nazwa, zm in ProdOkien.constraints.items()]
    print(pd.DataFrame(tabWrazliwosci))

interact(wyznOptymPraceFabryk,
    ZyskOkDrew = suw_zysk_drew, ZyskOkAlu = suw_zysk_alu,
    PracaF1Drew = suw_fabr_1_drew, PracaF1Alu = suw_fabr_1_alu,
    PracaF2Drew = suw_fabr_2_drew, PracaF2Alu = suw_fabr_2_alu,
    PracaF3Drew = suw_fabr_3_drew, PracaF3Alu = suw_fabr_3_alu,
    DostepF1 = suw_dostep_fabr_1, DostepF2 = suw_dostep_fabr_2, DostepF3 = suw_dostep_fabr_3)

interactive(children=(IntSlider(value=5000, description='Zysk (drew.) [zł]', max=10000, readout_format='.0f', …

<function __main__.wyznOptymPraceFabryk>

Parametr <code>LpProblem.contraints.values.slack</code> określa odległość danej wielkości od ograniczenia. Wartość zerowa oznacza zadziałanie ograniczenia, a więc osłabienie tego ograniczenia może poprawić wynik.
W tym przypadku wyraźnie widać, która fabryka ogranicza produkcję bardziej opłacalnej wersji okien.

Z kolei parametr <code>LpProblem.contraints.values.pi</code> określa wpływ danego ograniczenia na wynik. Dzięki temu można określić najbardziej kluczowy element procesu. Opcja ta nie działa poprawnie dla zmiennych decyzyjnych typu <code>LpInteger</code>.